In [2]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt


np.set_printoptions(threshold=sys.maxsize)
# Load the pickle file into a DataFrame
data = pd.read_pickle('ad-chega.pkl')

print(data.shape)
# Display the DataFrame~
#bottle, tie, person, tv, red wine, umbrella, train, 
#print(data.iloc[1,0])
print(np.array(data.iloc[:,1]))
#print(np.array(data.iloc[1,1]).shape)

#print(data.iloc[6,0])



Matplotlib is building the font cache; this may take a moment.


(2367, 7)
[list([[0, 53, 620, 659, 'person', 0.93359375], [612, 44, 646, 660, 'person', 0.91796875], [1084, 486, 174, 233, 'person', 0.5234375]])
 list([[87, 45, 740, 587, 'person', 0.94140625], [1069, 481, 195, 233, 'person', 0.87890625], [455, 464, 76, 155, 'tie', 0.61328125]])
 list([[625, 31, 646, 686, 'person', 0.9375], [7, 38, 621, 673, 'person', 0.93359375], [271, 457, 82, 164, 'tie', 0.5234375], [896, 480, 107, 141, 'tie', 0.5]])
 list([[604, 101, 111, 133, 'person', 0.80078125], [154, 129, 177, 215, 'person', 0.75390625], [1074, 481, 177, 233, 'person', 0.73828125], [857, 137, 174, 220, 'person', 0.61328125]])
 list([[91, 29, 767, 660, 'person', 0.9375], [1051, 488, 207, 228, 'person', 0.83203125]])
 list([[138, 26, 712, 646, 'person', 0.91015625], [1076, 481, 181, 233, 'person', 0.87890625]])
 list([[373, 51, 754, 659, 'person', 0.89453125], [1045, 479, 215, 237, 'person', 0.7890625], [664, 519, 118, 100, 'tie', 0.5]])
 list([[1, 44, 633, 660, 'person', 0.94140625], [647, 23,

In [3]:
countPessoa = 0
countLocal = 0
countbottle = 0
countTv = 0
countWine = 0
countUmbrella = 0
countTrain = 0
countBowl = 0
countTable = 0
countChair = 0
countBooks = 0

unique_objects = set()

for i in range(0,2367):
        
        Data_objects = np.array(data.iloc[i,1])
        
        for j in range(0, len(Data_objects)):
               
               unique_objects.add(Data_objects[j][4])

               if Data_objects[j][4] == 'person':
                    countPessoa += 1
               elif Data_objects[j][4] == 'tie':
                    countLocal += 1
               elif Data_objects[j][4] == 'bottle':
                    countbottle += 1
               elif Data_objects[j][4] == 'tv':
                    countTv += 1
               elif Data_objects[j][4] == 'wine glass':
                    countWine += 1
               elif Data_objects[j][4] == 'umbrella':
                    countUmbrella += 1
               elif Data_objects[j][4] == 'train':
                    countTrain += 1
               elif Data_objects[j][4] == 'bowl':
                    countBowl += 1
               elif Data_objects[j][4] == 'dining table':
                    countTable += 1
               elif Data_objects[j][4] == 'chair':
                    countChair += 1
               elif Data_objects[j][4] == 'book':
                    countBooks += 1
               
print(countPessoa)
print(countLocal)
print(countbottle)
print(countTv)
print(countWine)
print(countUmbrella)
print(countTrain)
print(countBowl)
print(countTable)
print(countChair)
print(countBooks)


                                
                        
      



    

5466
1587
33
31
2
2
2
1
1
1
1
